<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

## Step 1.1: Configure the environment

In [ ]:
from datetime import datetime
import geopandas as gpd
import numpy as np
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
# Predefined variable for title and version

# Enter your initials
initials = "ja"

# site
site = "tongatapu"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{site}-{date.strftime('%d%m%Y')}"
print(version)

### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [ ]:
joined_df = gpd.read_file(f"training-data/{version}-training.csv")
joined_df = joined_df.astype("float32")
joined_df

In [ ]:
joined_df=joined_df.drop(columns=["y", "x"])

In [ ]:
print(len(joined_df.columns))
joined_df.columns

In [ ]:
from sklearn.model_selection import train_test_split

training_data, test_data = train_test_split(joined_df, test_size=0.2, random_state=1337)

# The classes are the first column
classes = np.array(training_data)[:, 0]

# The observation data is everything after the second column
observations = np.array(training_data)[:, 1:]

# Create a model...
classifier = RandomForestClassifier(max_depth=4)

# ...and fit it to the data
model = classifier.fit(observations, classes)

In [ ]:
# Dynamically create the filename with f-string
file_path = f"models/{version}-test.model"

# Save the model
joblib.dump(model, file_path)

In [ ]:
import pandas as pd
test_actual = np.array(test_data)[:, 0]
test_predicted = model.predict(np.array(test_data)[:,1:])
pd.crosstab(test_actual, test_predicted, margins=True)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(test_actual, test_predicted)